In [1]:
import requests
import pandas as pd
from datetime import datetime
import psycopg2
import time

In [2]:
conn = psycopg2.connect(database="postgres", user="postgres",
                        password="Aa@123456", host="127.0.0.1", port="5432")

In [13]:
cur = conn.cursor()
cur.execute('''SELECT created_at, recommendation, price FROM Recommendations;''')
rows = cur.fetchall()
conn.commit()

if rows[0][1] == "SELL":
    rows = rows[1:]
    
if rows[len(rows) -1] == "BUY":
    rows = rows[:-1]

In [14]:
df = pd.DataFrame(rows, columns=['time', 'recommendation', 'price'])

In [22]:
# Investment
amount= 100000
transactions_num = len(df)
transactions_fees = 0.5 # maybe we need to use if statements here
total_fees = transactions_num * transactions_fees

In [23]:
df["pct"]= 0
df["runing_amount"]= 0

In [24]:
df

,time,recommendation,price,pct,runing_amount
0,2021-06-30 23:44:30,BUY,34984.87915,0,0
1,2021-06-30 23:45:30,SELL,34980.27085,0,0
2,2021-06-30 23:47:30,BUY,34950.87665,0,0
3,2021-06-30 23:51:30,SELL,34900.28000,0,0


In [29]:
x=0

for i in range(len(df)):
    if df["recommendation"].iloc[x] == "SELL":
        df["pct"].iloc[i]=(df["price"].iloc[i]-df["price"].iloc[i-1])/df["price"].iloc[i-1]
        if x<=1:
            df["runing_amount"].iloc[i]=(1+df["pct"].iloc[i])*amount
        else:
            df["runing_amount"].iloc[i]=(1+df["pct"].iloc[i])*df["runing_amount"].iloc[i-2]

In [30]:
rtn_pct=(df["runing_amount"].iloc[-1])/ amount * 100
#dollar profit/loss
profit_loss= df["runing_amount"].iloc[-1] - amount

In [31]:
print(rtn_pct,"% \n",profit_loss,"\n")

99.84208176827468 % 
 -157.9182317253144 



In [ ]:
@btcData.route("/btc/profit", methods=["GET"])
def get_btc_profit():
    res = {}
    cur = conn.cursor()
    cur.execute('''SELECT created_at, recommendation, price FROM Recommendations;''')
    rows = cur.fetchall()
    conn.commit()

    if rows[0][1] == "SELL":
        rows = rows[1:]

    if rows[len(rows) -1] == "BUY":
        rows = rows[:-1]
        
    df = pd.DataFrame(rows, columns=['time', 'recommendation', 'price'])
    df["pct"]= 0
    df["runing_amount"]= 0
    
    # Investment
    amount= 100000
    transactions_num = len(df)
    transactions_fees = 0.5 # maybe we need to use if statements here
    total_fees = transactions_num * transactions_fees
    
    for i in range(len(df)):
        if df["recommendation"].iloc[x] == "SELL":
            df["pct"].iloc[i]=(df["price"].iloc[i]-df["price"].iloc[i-1])/df["price"].iloc[i-1]
            if x<=1:
                df["runing_amount"].iloc[i]=(1+df["pct"].iloc[i])*amount
            else:
                df["runing_amount"].iloc[i]=(1+df["pct"].iloc[i])*df["runing_amount"].iloc[i-2]
    
    rtn_pct=(df["runing_amount"].iloc[-1])/ amount * 100
    
    # dollar profit/loss
    profit_loss= df["runing_amount"].iloc[-1] - amount
    
    res["rtn_pct"] = rtn_pct
    res["profit_loss"] = profit_loss
    
    result = jsonify(res)
    result.headers.add('Access-Control-Allow-Origin', '*')

    return result